## Histone deacetylase 1 - part 4 (library enumeration)

### Import libraries

In [1]:
import pandas as pd

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

from rdkit import Chem

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


### Read in the datasets

In [2]:
df = pd.read_csv('hdac1_inhibitors_stripped.csv')
df_trans = pd.read_csv('hdac1_inhibitors_transformations.csv')

### Merge on assay identifier to enumerate all seed molecule / transformation pairs

In [3]:
df = df[pd.notna(df.pchembl_value)]
df = df.merge(df_trans, on='assay_chembl_id')

### Apply the transformations

In [4]:
def apply_reaction(smiles, smirks):
    rxn = Chem.rdChemReactions.ReactionFromSmarts(smirks)
    productset = rxn.RunReactants((Chem.AddHs(Chem.MolFromSmiles(smiles)),))
    productlist = list()
    for product in productset:
        productlist.append('.'.join([Chem.MolToSmiles(Chem.RemoveHs(productpart)) for productpart in product]))
    return list(set(productlist))

df_seed = df.parallel_apply(lambda x: apply_reaction(x.stripped_smiles, x.smirks), axis=1)

### Associate products with seed data

In [5]:
df_seed = df.join(pd.DataFrame(df_seed.explode(), columns=['products']))
df_seed = df_seed[pd.notna(df_seed.products)]

### Seed + delta = prediction, and aggregate (optionally weighted according to count of pairs in transformation)

In [6]:
df_seed['pchembl_predicted'] = (df_seed.pchembl_value + df_seed.mean_pchembl_exact_delta) #* df_seed.count_pchembl_exact_delta
df_products = pd.pivot_table(df_seed, values=['count_pchembl_exact_delta', 'pchembl_predicted'], index=['products', 'radius', 'assay_chembl_id'], aggfunc='mean')
#df_products['pchembl_predicted'] = df_products.pchembl_predicted / df_products.count_pchembl_exact_delta

### Write to file

In [7]:
df_products = df_products.reset_index()
df_products.to_csv('hdac1_inhibitors_products.csv', index=False)

### ... and finaly tabulate (tables 4 & 5 from wizepairz paper)

In [8]:
df_holdout = pd.read_csv('hdac1_inhibitors_stripped_holdout.csv')
df_holdout = df_seed.merge(df_holdout, left_on=['assay_chembl_id', 'products'], right_on=['assay_chembl_id', 'stripped_smiles'])
tablecols = ['_metadata.parent_molecule_data.compound_key_x', 'fragment1', 'fragment2', 'pchembl_value_x', 'mean_pchembl_exact_delta', 'pchembl_predicted']

In [9]:
df_holdout[(df_holdout.radius==3) & (df_holdout.assay_chembl_id=='CHEMBL927948') & (df_holdout['_metadata.parent_molecule_data.compound_key_y']=='13c')][tablecols]

,_metadata.parent_molecule_data.compound_key_x,fragment1,fragment2,pchembl_value_x,mean_pchembl_exact_delta,pchembl_predicted
1,13b,66c735f6aa9c5b5cc24a7258604abfe3,709d9861baef05140bf108058786fabf,8.01,-0.16,7.85
4,13a,1e5e8361c8f05f9f512e901d22506931,709d9861baef05140bf108058786fabf,7.67,0.11,7.78
11,13d,f023e31e01c29c216d13ecf4ae323f3b,709d9861baef05140bf108058786fabf,7.30,0.40,7.70


In [10]:
df_holdout[(df_holdout.radius==3) & (df_holdout.assay_chembl_id=='CHEMBL927948') & (df_holdout['_metadata.parent_molecule_data.compound_key_y']=='14d')][tablecols]

,_metadata.parent_molecule_data.compound_key_x,fragment1,fragment2,pchembl_value_x,mean_pchembl_exact_delta,pchembl_predicted
15,15d,a545b6dcb46a2e453d6a86e729aa3e08,1fd8ece99712a9432eb8759959e992c6,7.48,-0.016667,7.463333
16,14a,1e5e8361c8f05f9f512e901d22506931,f023e31e01c29c216d13ecf4ae323f3b,7.73,-0.305000,7.425000
21,14b,66c735f6aa9c5b5cc24a7258604abfe3,f023e31e01c29c216d13ecf4ae323f3b,8.01,-0.605000,7.405000
24,14c,709d9861baef05140bf108058786fabf,f023e31e01c29c216d13ecf4ae323f3b,7.79,-0.400000,7.390000
